In [1]:
from hypergraphs import *
from kegg import *

In [2]:
from bs4 import BeautifulSoup 
import pandas as pd

In [3]:
def getSampleAges(filename):
    res = {}
    with open(filename, "r") as file:
        data = file.read()
    bs_data = BeautifulSoup(data, 'xml') 
    samples = bs_data.find_all('Sample') 
    for sample in samples:
        identifier = str(sample.find('Title').string).strip()
        ages = sample.find('Characteristics', {'tag': 'age'})
        res[identifier]=str(ages.string).strip()
    return res

In [4]:
sample_ages = getSampleAges("../../LaTeX/Data/KEGG-Pathways/AgeDeregulation/mmu_1.xml")

In [6]:
def getGeneCount(filename, sheet_name, sample_column_names, gene_id_column):
    df = pd.read_excel(filename, sheet_name = sheet_name)
    df.rename(columns = {x: x.strip() for x in df.columns}, inplace=True)
    gene_ids = df.loc[:, gene_id_column]
    return gene_ids, {sample_column_name: df.loc[:, sample_column_name] 
            for sample_column_name in sample_column_names
            if sample_column_name in df.columns},
        

In [7]:
gn, df = getGeneCount("../../LaTeX/Data/KEGG-Pathways/AgeDeregulation/mmu_0.xls", "counts",  sample_ages.keys(), "external_gene_id")

In [8]:
gn

0              Gnai3
1               Pbsn
2              Cdc45
3                H19
4              Scml2
            ...     
37310     AC158600.6
37311    AC169674.49
37312     AC116875.1
37313         Gm2964
37314     AC158220.1
Name: external_gene_id, Length: 37315, dtype: object